# Data Imputation

### Libs

In [ ]:
## libs 
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

## Keras
from keras.layers import Lambda, Input, Dense, Conv2D, Conv2DTranspose, Flatten, Reshape
from keras.models import Model
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.losses import mse, binary_crossentropy
from keras.utils import plot_model
from keras import backend as K

## Basic
from tqdm import tqdm_notebook as tqdm
import argparse
import os
import random
import itertools
import time

# Computation
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mutual_info_score
from sklearn.preprocessing import MinMaxScaler

import scipy
from scipy.stats.stats import pearsonr 
import pandas as pd

## Visualization
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns

## Network Processing
import networkx as nx
from networkx.generators import random_graphs

## node colour
orig_cmap = plt.cm.PuBu

### Supporting Function

In [ ]:
## supporting functions
from support.preprocessing import sort_adj, reshape_A, calculate_A_shape, reconstruct_adjacency, pad_matrix, unpad_matrix, pad_attr, unpad_attr, prepare_in_out
from support.metrics import compute_mig, compute_mi
from support.graph_generating import generate_single_features, generate_manifold_features
from support.latent_space import vis2D, visDistr
from support.comparing import compare_manifold_adjacency, compare_topol_manifold
from support.plotting import shiftedColorMap

## graph sampling
from sampling import ForestFire, Metropolis_Hastings, Random_Walk, Snowball, Ties, Base_Samplers

In [ ]:
def sort_adj(g):
    node_k1 = dict(g.degree())  # sort by degree
    node_k2 = nx.average_neighbor_degree(g)  # sort by neighbor degree
    node_closeness = nx.closeness_centrality(g)
    node_betweenness = nx.betweenness_centrality(g)

    node_sorting = list()

    ## sort topology amd attributes _________________

    # for node_id in range(0, len(g)):
    for node_id in g.nodes():
        node_sorting.append(
            (node_id, node_k1[node_id], node_k2[node_id], node_closeness[node_id], node_betweenness[node_id]))

    node_descending = sorted(node_sorting, key=lambda x: (x[1], x[2], x[3], x[4]), reverse=True)

    mapping = dict()
    # f_sorted = f

    for i, node in enumerate(node_descending):
        mapping[node[0]] = i
        # f_sorted[i] = node_descending[i][5]

    a = nx.adjacency_matrix(g, nodelist=mapping.keys()).todense()

    return g, a

### Load Real-World Graph Data

In [ ]:
g_complete = nx.read_edgelist("/Users/niklasstoehr/Programming/thesis/4_real_attr/data/mag/edges_py.csv",  nodetype=int, delimiter = ",")
#a_complete = nx.adjacency_matrix(g_complete)
a_complete = None

df_nodes  = pd.read_csv("/Users/niklasstoehr/Programming/thesis/4_real_attr/data/mag/nodes_py.csv", header = None)
nodes = df_nodes.values
nodes = [l.tolist() for l in list(nodes)]
nodes_num = [i[0] for i in nodes]

g_complete.add_nodes_from(nodes_num)
node_attr_dict = dict()

for num, name, c in nodes:
    node_attr_dict[num] = c
        
nx.set_node_attributes(g_complete, node_attr_dict, "citations")

In [ ]:
sampleArgs = {"sample": "biased_random_walk", "jump_bias": "random_walk_induced_graph_sampling", "n": 10, "p": 20.0, "q": 100.0, "source_starts": 2, "source_returns": 4, "depth": 2}


def get_graph(sampleArgs,g_complete,a_complete):
    
    if sampleArgs["sample"] == "biased_random_walk":
        sampler = Base_Samplers.Base_Samplers(g_complete,a_complete)
        #sampler = Base_Samplers(g_complete,a_complete)
        g = sampler.biased_random_walk(sampleArgs["n"], sampleArgs["p"], sampleArgs["q"])

    if sampleArgs["sample"] == "forestfire":
        sampler = ForestFire.ForestFire(g_complete,a_complete)
        g = sampler.forestfire(sampleArgs["n"])

    if sampleArgs["sample"] == "snowball":
        sampler = Snowball.Snowball(g_complete,a_complete)
        g = sampler.snowball(sampleArgs["source_starts"], sampleArgs["source_returns"])

    if sampleArgs["sample"] == "random_walk_induced_graph_sampling":
        sampler = Random_Walk.Random_Walk(g_complete,a_complete)
        g = sampler.random_walk_induced_graph_sampling(sampleArgs["n"])

    if sampleArgs["sample"] == "random_walk_sampling_with_fly_back":
        sampler = Random_Walk.Random_Walk(g_complete,a_complete)
        g = sampler.random_walk_sampling_with_fly_back(sampleArgs["n"], sampleArgs["p"])
        
    if sampleArgs["sample"] == "standard_bfs":
        sampler = Base_Samplers.Base_Samplers(g_complete,a_complete)
        g = sampler.standard_bfs(sampleArgs["source_starts"], sampleArgs["depth"]) 
        
    if sampleArgs["sample"] == "bfs":
        sampler = Base_Samplers.Base_Samplers(g_complete,a_complete)
        g = sampler.bfs(sampleArgs["n"]) 
        
    if sampleArgs["sample"] == "walk":
        sampler = Base_Samplers.Base_Samplers(g_complete,a_complete)
        g = sampler.walk(sampleArgs["source_starts"], sampleArgs["source_returns"], sampleArgs["p"])        
        
    if sampleArgs["sample"] == "jump":
        sampler = Base_Samplers.Base_Samplers(g_complete,a_complete)
        g = sampler.jump(sampleArgs["source_starts"], sampleArgs["p"], sampleArgs["jump_bias"])
        
    if sampleArgs["sample"] == "adjacency":
        sampler = Base_Samplers.Base_Samplers(g_complete,a_complete)
        g = sampler.adjacency(sampleArgs["n"]) 
        
    if sampleArgs["sample"] == "select":
        sampler = Base_Samplers.Base_Samplers(g_complete,a_complete)
        g = sampler.adjacency(sampleArgs["n"]) 
    
    return g 

g = get_graph(sampleArgs,g_complete,a_complete)

In [ ]:
dataArgs = {"n_graphs": 100, "n_max": 12, "fix_n": False, "diag_offset": -2, "diag_value": 1, "clip": True, "n_features": 1}  #"diag_offset" - 1 == full adjacency

def generate_data(dataArgs): 
    
    ## Data ________________________________

    G = np.zeros((dataArgs["n_graphs"], *calculate_A_shape(dataArgs["n_max"], dataArgs["diag_offset"])))
    F = np.zeros((dataArgs["n_graphs"], dataArgs["n_max"], dataArgs["n_features"]))    
    
    ## Generate Graph Data_______________________________

    for i in tqdm(range(0,dataArgs["n_graphs"])):
        
        ## Generate Graph Type ______________________________________________

        g = get_graph(sampleArgs,g_complete,a_complete)
        
        g, a = sort_adj(g)
        a = pad_matrix(a, dataArgs["n_max"], dataArgs["diag_value"])  # pad adjacency matrix to allow less nodes than n_max and fill diagonal        
        a_transformed = reshape_A(a, diag_offset = dataArgs["diag_offset"])
        
        
        ## Generate / Load Node Features ______________________________________________
        f = np.asarray([node_attr_dict[x] for x in list(g.nodes())])
        #f = np.asarray(nx.get_node_attributes(g_complete, "citations"))
        
        ## pad features with zeroes
        f = np.reshape(f, (f.shape[0], 1))
        f = pad_attr(f, dataArgs)

        
        ## Build Data Arrays___________________________________________________

        F[i] = f
        G[i] = a_transformed


    ## Input and Output Size ___________________________________________________________

    input_shape, output_shape = prepare_in_out(dataArgs["diag_offset"], calculate_A_shape(dataArgs["n_max"], dataArgs["diag_offset"]), F.shape)
    print("input_shape:", input_shape, ", output_shape:", output_shape)
    
    ## scale features in F for smoother training
    #scaler = MinMaxScaler()
    #scaler.fit(F)
    #F = scaler.transform(F)
    
    return G, F, input_shape,output_shape
    
G, F, input_shape, output_shape = generate_data(dataArgs)

## Data Imputation

In [ ]:
def generate_features(dataArgs, g, n):
    
        if dataArgs["feature_dependence"] == "random":
            f = np.random.rand(n, dataArgs["n_features"])                   ## float
            #F[i] = np.random.randint(2, size=(dataArgs["n_max"],dataArgs["n_features"]))   ## int
            
        if dataArgs["feature_dependence"] == "norm_degree":
            if dataArgs["n_features"] == 1:
                
                f = np.asarray([int(x[1]) for x in sorted(g.degree())])  
                f = (f) / (max(f)+1)
                f = np.reshape(f, (f.shape[-1],1))
                
                    
        if dataArgs["feature_dependence"] == "degree":
            if dataArgs["n_features"] == 1:
                
                f = np.asarray([int(x[1]) for x in sorted(g.degree())])  
                f = (f+1) / (dataArgs["n_max"]+1)
                f = np.reshape(f, (f.shape[-1],1))
                
                
        return f

In [ ]:
def imput_data(imputeArgs, modelArgs, dataArgs, G, F):
        
    G_imputed = np.copy(G)
    F_imputed = np.copy(F)
    
    if imputeArgs["impute"] == "features":
        
        for i, (g,f) in enumerate(zip(G_imputed, F_imputed)):
            
            f = np.squeeze(f)
            num_features = len(f[f > 0])
            impute_num = int(imputeArgs["impute_frac"] * num_features)
            impute_f_ind = random.sample(range(num_features), impute_num)  ## impute features

            row, col = np.diag_indices(f.shape[0])

            f[impute_f_ind] = imputeArgs["impute_value"]  ## replace features
            F_imputed[i] = np.reshape(f,(f.shape[-1], 1))
        
        
    
    if imputeArgs["impute"] == "structure":
    
        for i, (g,f) in enumerate(zip(G_imputed, F_imputed)):
                        
            f = np.squeeze(f)            
            num_nodes = len(f[f > 0])
            impute_num = int(imputeArgs["impute_frac"] * num_nodes)
            impute_n_ind = random.sample(range(num_nodes), impute_num)  ## impute nodes
            
            ## remove edges of imputed nodes
            for impute_n in impute_n_ind:
                
                ## remove rows
                g[impute_n,:impute_n] = 0
                g[impute_n,impute_n+1:] = 0
                
                ## remove columns
                g[:impute_n,impute_n] = 0
                g[impute_n+1:,impute_n] = 0

            G_imputed[i] = g
        
    return G_imputed, F_imputed

# beta-VAE Model

In [ ]:
## Model Setup
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

## Keras
from keras.layers import Lambda, Input, Dense, Conv2D, Conv2DTranspose, Flatten, Reshape, Dropout, Activation, concatenate
from keras.models import Model
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.losses import mse, binary_crossentropy
from keras.utils import plot_model
from keras import backend as K
import tensorflow as tf

from sklearn.model_selection import train_test_split
from support.keras_dgl.utils import *
from support.keras_dgl.layers import MultiGraphCNN


class VAE():

    # reparameterization trick
    # instead of sampling from Q(z|X), sample eps = N(0,I)
    # then z = z_mean + sqrt(var)*eps

    def sampling(self, args):
        """Reparameterization trick by sampling fr an isotropic unit Gaussian.
        # Arguments
            args (tensor): mean and log of variance of Q(z|X)
        # Returns
            z (tensor): sampled latent vector
        """

        z_mean, z_log_var = args
        batch = K.shape(z_mean)[0]
        dim = K.int_shape(z_mean)[1]
        # by default, random_normal has mean=0 and std=1.0
        epsilon = K.random_normal(shape=(batch, dim))
        return z_mean + K.exp(0.5 * z_log_var) * epsilon

    
    
    def __init__(self, modelArgs, trainArgs, g_train, g_train_mod_imputed, g_test, g_test_mod_imputed, f_train, f_train_imputed, f_test, f_test_imputed):

        ## MODEL ______________________________________________________________       
            
        ## Graph Neural Network Architecture __________________________________
            
        ## 1) build encoder model____________________________________

        # build graph_conv_filters
        SYM_NORM = True
        num_filters = modelArgs['gnn_filters']
        graph_conv_filters = preprocess_adj_tensor_with_identity(np.squeeze(g_train), SYM_NORM)

        # build model
        X_input = Input(shape=(f_train.shape[1], f_train.shape[2]))
        graph_conv_filters_input = Input(shape=(graph_conv_filters.shape[1], graph_conv_filters.shape[2]))

        # define inputs of features and graph topologies
        inputs = [X_input, graph_conv_filters_input]

        x = MultiGraphCNN(100, num_filters, activation='elu')([X_input, graph_conv_filters_input])
        x = Dropout(0.2)(x)
        x = MultiGraphCNN(100, num_filters, activation='elu')([x, graph_conv_filters_input])
        x = Dropout(0.2)(x)
        x = Lambda(lambda x: K.mean(x, axis=1))(x)  # adding a node invariant layer to make sure output does not depends upon the node order in a graph.
        x = Dense(8, activation='relu')(x)
        z_mean = Dense(modelArgs["latent_dim"], name='z_mean')(x)
        z_log_var = Dense(modelArgs["latent_dim"], name='z_log_var')(x)

        # use reparameterization trick to push the sampling out as input
        # note that "output_shape" isn't necessary with the TensorFlow backend
        z = Lambda(self.sampling, output_shape=(modelArgs["latent_dim"],), name='z')([z_mean, z_log_var])
        
        latent_inputs = Input(shape=(modelArgs["latent_dim"],), name='z_sampling')

        
        ## 2.1) build attribute decoder model __________________________

        y = Dense(4, activation='relu')(latent_inputs)
        y = Dense(6, activation='relu')(latent_inputs)
        y = Dense(10, activation='relu')(latent_inputs)
        y = Dense(modelArgs["output_shape"][0][0], activation='sigmoid')(y)
        attr_output = Reshape(modelArgs["output_shape"][0])(y)
        
        
        
        ## 2.2) build topology decoder model __________________________

        ## shape info needed to build decoder model
        inputs_2D_encoder = Input(shape=modelArgs["input_shape"][1], name='encoder_input')
        x_2D = inputs_2D_encoder
        for i in range(2):
            modelArgs['conv_filters'] *= 2
            x_2D = Conv2D(filters=modelArgs['conv_filters'], kernel_size=modelArgs['kernel_size'], activation='relu',strides=2, padding='same')(x_2D)
        shape_2D = K.int_shape(x_2D)

        x_2D = Dense(shape_2D[1] * shape_2D[2] * shape_2D[3], activation='relu')(latent_inputs)
        x_2D = Reshape((shape_2D[1], shape_2D[2], shape_2D[3]))(x_2D)

        for i in range(2):
            x_2D = Conv2DTranspose(filters=modelArgs['conv_filters'], kernel_size=modelArgs['kernel_size'],activation='relu', strides=2, padding='same')(x_2D)
            modelArgs['conv_filters'] //= 2

        topol_output = Conv2DTranspose(filters=1, kernel_size=modelArgs['kernel_size'], activation='sigmoid',padding='same', name='decoder_output')(x_2D)



     
    
        ## INSTANTIATE___________________________________

        ## 1) instantiate joint encoder model
        encoder = Model(inputs, [z_mean, z_log_var, z], name='encoder')
        encoder.summary()

        ## 2) instantiate topology decoder model
        decoder = Model(latent_inputs, [attr_output, topol_output], name='decoder')
        decoder.summary()
        
        ## 3) instantiate VAE model
        attr_topol_outputs = decoder(encoder(inputs)[2])
        vae = Model(inputs, attr_topol_outputs, name='vae')

    
    

        ## LOSS FUNCTIONS ______________________________________
        
        def loss_func(y_true, y_pred):
            
            y_true_attr = y_true[0]
            y_pred_attr = y_pred[0]
            
            y_true_topol = y_true[1]
            y_pred_topol = y_pred[1]            

            ## ATTR RECONSTRUCTION LOSS_______________________            
            ## mean squared error
            attr_reconstruction_loss = mse(K.flatten(y_true_attr), K.flatten(y_pred_attr))
            attr_reconstruction_loss *= modelArgs["input_shape"][0][0]
            
            ## TOPOL RECONSTRUCTION LOSS_______________________
            ## binary cross-entropy
            topol_reconstruction_loss = binary_crossentropy(K.flatten(y_true_topol), K.flatten(y_pred_topol))
            topol_reconstruction_loss *= (modelArgs["input_shape"][1][0] * modelArgs["input_shape"][1][1])
                     
            ## KL LOSS _____________________________________________
            kl_loss = 1 + z_log_var - K.square(z_mean) - K.exp(z_log_var)
            kl_loss = K.sum(kl_loss, axis=-1)
            kl_loss *= -0.5

            ## COMPLETE LOSS __________________________________________________

            loss = K.mean(trainArgs["loss_weights"][0] * attr_reconstruction_loss + trainArgs["loss_weights"][1] * topol_reconstruction_loss + trainArgs["loss_weights"][2] * kl_loss)
            
            return loss
        
    
    
    
        ## MODEL COMPILE______________________________________________
        
        vae.compile(optimizer='adam', loss=loss_func)
        vae.summary()
        
        

        ## TRAIN______________________________________________

        # load the autoencoder weights

        if trainArgs["weights"] == "load":

            vae.load_weights("models/weights/vae_mlp_mnist_latent_dim_" + str(modelArgs["latent_dim"]) + ".h5")

        # train the autoencoder

        elif trainArgs["weights"] == "train":

            # Set callback functions to early stop training and save the best model so far
            callbacks = [EarlyStopping(monitor='val_loss', patience=trainArgs["early_stop"]), ModelCheckpoint(filepath="models/weights/vae_mlp_mnist_latent_dim_" + str(modelArgs["latent_dim"]) + ".h5",save_best_only=True)]

            vae.fit([f_train_imputed, g_train_mod_imputed], [f_train, g_train], epochs=trainArgs["epochs"],batch_size=trainArgs["batch_size"], callbacks=callbacks,validation_data=([f_test_imputed, g_test_mod_imputed], [f_test, g_test]))
            vae.save_weights("models/weights/vae_mlp_mnist_latent_dim_" + str(modelArgs["latent_dim"]) + ".h5")

            self.model = (encoder, decoder)
            

### Data Split and Impute

In [ ]:
trainArgs = {"loss_weights": [100,1,5], "weights": "train", "early_stop": 1, "batch_size": 128, "epochs": 50, "data_split": 0.2}
modelArgs = {"nn_architecture": "gnn", "latent_dim": 2, "gnn_filters": 2, "conv_filters": 16, "kernel_size": 3, "input_shape": input_shape, "output_shape": output_shape, "param_loss": False,}
imputeArgs = {"impute": "features", "impute_frac": 1.0, "impute_value": 0.0, }

from support.keras_dgl.utils import *
from support.keras_dgl.layers import MultiGraphCNN

## Train and Validation Split _______________________________________________
g_train, g_test, f_train, f_test = train_test_split(G, F, test_size=trainArgs["data_split"], random_state=1, shuffle=True)

## impute the data
g_train_imputed, f_train_imputed = imput_data(imputeArgs, modelArgs, dataArgs, g_train, f_train)
g_test_imputed, f_test_imputed = imput_data(imputeArgs, modelArgs, dataArgs, g_test, f_test)

# build graph_conv_filters
SYM_NORM = True
g_train_mod_imputed = preprocess_adj_tensor_with_identity(np.squeeze(g_train_imputed), SYM_NORM)
g_test_mod_imputed = preprocess_adj_tensor_with_identity(np.squeeze(g_test_imputed), SYM_NORM)

data = (f_test_imputed, [g_test_mod_imputed, g_test])

In [ ]:
vae = VAE(modelArgs, trainArgs, g_train, g_train_mod_imputed, g_test, g_test_mod_imputed, f_train, f_train_imputed, f_test, f_test_imputed)

models = vae.model 

# Analysis

### Generate Single Graph

In [ ]:
analyzeArgs = {"z": [0,1], "activations": [20,-2], "normalize_feature": False}
generate_single_features(analyzeArgs, modelArgs, dataArgs, models, orig_cmap)

## Impaint Graphs (Extreme Cases)

In [ ]:
def impaint(analyzeArgs, imputeArgs, modelArgs, models, batch_size=128):

    print("impainting the", imputeArgs["impute"])
    
    ## unpack models__________________________
    
    encoder, decoder = models  # trained models
    
    ## generate feature data___________________________
    f = np.reshape([analyzeArgs["f"]] * analyzeArgs["n"], (analyzeArgs["n"], 1))
    
    ## pad features with zeroes
    f = pad_attr(f, dataArgs)

    
    ## generate graph data___________________________
    G = np.zeros((1, *calculate_A_shape(dataArgs["n_max"], dataArgs["diag_offset"])))
    F = np.zeros((1, dataArgs["n_max"], dataArgs["n_features"]))

    g = get_graph(sampleArgs,g_complete,a_complete)
    
    g, a = sort_adj(g)
    a = pad_matrix(a, dataArgs["n_max"], dataArgs["diag_value"])  # pad adjacency matrix to allow less nodes than n_max and fill diagonal
    a = reshape_A(a, diag_offset = dataArgs["diag_offset"])
    
    G[0] = a
    F[0] = f
    
    
    ## impute data_______________________
    
    a_imputed, f_imputed = imput_data(imputeArgs, modelArgs, dataArgs, G, F)
    
    
    ## ENCODER_________________________________
    
    # build graph_conv_filters
    SYM_NORM = True
    a_imputed_mod = preprocess_adj_tensor_with_identity(np.squeeze(a_imputed), SYM_NORM)
    z_mean, _, _ = encoder.predict([f_imputed, a_imputed_mod], batch_size = batch_size)

    
    ## DECODER_________________________________    
    
    [f_decoded, a_decoded] = decoder.predict(z_mean)
    a_decoded = np.squeeze(a_decoded[0])
    f_decoded = f_decoded[0]
    
    
    ## GRAPH RECONSTRUCTION______________________
    
    ## reconstruct graph from output
    reconstructed_a = reconstruct_adjacency(a_decoded, dataArgs["clip"], dataArgs["diag_offset"])
    reconstructed_a, n_nodes = unpad_matrix(reconstructed_a, dataArgs["diag_value"], 0.2, dataArgs["fix_n"])
    reconstructed_g = nx.from_numpy_matrix(reconstructed_a)
    
    ## reconstruct attributes
    reconstructed_f = unpad_attr(f_decoded, n_nodes, analyzeArgs, dataArgs)
    
    ## create imputed graph
    a_imputed, n_nodes = unpad_matrix(np.squeeze(a_imputed[0]), dataArgs["diag_value"], 0.2, dataArgs["fix_n"])
    g_imputed = nx.from_numpy_matrix(a_imputed)
    
    f_imputed = np.reshape(f_imputed, (f_imputed.shape[1]))
    f_imputed = f_imputed[:analyzeArgs["n"]]
    
    print("original attributes:", f_imputed)
    
    
    ## GRAPH DRAWING_____________________________
    
    ## 1) draw imputed graph
    if reconstructed_f.shape[0] > 0:
        fixed_cmap = shiftedColorMap(orig_cmap, start=min(f_imputed), midpoint=0.5, stop=max(f_imputed),name='fixed')
    else:
        fixed_cmap = shiftedColorMap(orig_cmap, start=0.5, midpoint=0.5, stop=0.5, name='fixed')    
    nx.draw(g_imputed, node_color=f_imputed, font_color='white', cmap = fixed_cmap)
    plt.show()
    
    ## 2) draw reconstructed graph
    if reconstructed_f.shape[0] > 0:
        fixed_cmap = shiftedColorMap(orig_cmap, start=min(reconstructed_f), midpoint=0.5, stop=max(reconstructed_f),name='fixed')
    else:
        fixed_cmap = shiftedColorMap(orig_cmap, start=0.5, midpoint=0.5, stop=0.5, name='fixed') 
    print("reconstructed attributes::", reconstructed_f)
    nx.draw(reconstructed_g, node_color=reconstructed_f, font_color='white', cmap = fixed_cmap)
    plt.show()

        
  ## PLOT RESULTS ________________________________________

imputeArgs["impute_frac"] = 1.0
analyzeArgs = {"n": 10, "p": 1.0, "f": 0.4, "normalize_feature": False}
impaint(analyzeArgs, imputeArgs, modelArgs, models, batch_size=trainArgs["batch_size"])